In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

In [3]:
transform = transforms.Compose([
        transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root = "./data", train = True, download = True, transform = transform)
test_dataset = datasets.CIFAR10(root = "./data", train = False, download = True, transform = transform)

100%|██████████| 170498071/170498071 [01:06<00:00, 2567102.62it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))

train_loader = DataLoader(train_subset, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_subset, batch_size = 10, shuffle = False)

In [9]:
class CNNWithBNDropout(nn.Module):
        def __init__(self):
                super(CNNWithBNDropout, self).__init__()
                self.conv_block1 = nn.Sequential(
                        nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(2)
                )

                self.conv_block2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
                        nn.BatchNorm2d(64),
                        nn.ReLU(),
                        nn.MaxPool2d(2)
                )

                self.dense1 = nn.Linear(64 * 8 * 8, 512)
                self.dense2 = nn.Linear(512, 10)

                self.dropout = nn.Dropout(0.5)
                self.relu = nn.ReLU()
                self.flatten = nn.Flatten()

        def forward(self, x):
                x = self.conv_block1(x)
                x = self.conv_block2(x)
                x = self.flatten(x)
                x = self.dense1(x)
                x = self.relu(x)
                x = self.dense2(x)
                x = self.dropout(x)

                return x

model = CNNWithBNDropout()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


In [10]:
def train_model(num_epochs):
        for epoch in range(num_epochs):
                model.train()
                train_loss = 0
                correct_train = 0
                total_train = 0
                for data, target in train_loader:
                        optimizer.zero_grad()
                        op = model(data)
                        loss = criterion(op, target)
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item()
                        predicted = torch.argmax(op.data, dim = 1)
                        total_train += target.size(0)
                        correct_train += (predicted == target).sum().item()

                avg_train_loss = train_loss / len(train_loader)
                train_acc = 100 * correct_train / total_train

                model.eval()
                test_loss = 0
                correct_test = 0
                total_test = 0

                with torch.no_grad():
                        for data, target in test_loader:
                                op = model(data)
                                loss = criterion(op, target)
                                test_loss += loss.item()
                                predicted = torch.argmax(op.data, dim = 1)
                                total_test += target.size(0)
                                correct_test += (predicted == target).sum().item()
                
                avg_test_loss = test_loss / len(test_loader)
                test_acc = 100 * correct_test / total_test

                print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss: .4f}, Train Accuracy: {train_acc:.8f}%,'
                      f'Test Loss: {avg_test_loss: .4f}, Test Accuracy: {test_acc: .8f}%')
                

train_model(10)

Epoch 1, Train Loss:  3.8864, Train Accuracy: 11.00000000%,Test Loss:  2.4400, Test Accuracy:  18.00000000%
Epoch 2, Train Loss:  2.5766, Train Accuracy: 15.50000000%,Test Loss:  2.1506, Test Accuracy:  14.00000000%
Epoch 3, Train Loss:  2.1177, Train Accuracy: 24.50000000%,Test Loss:  2.1097, Test Accuracy:  24.00000000%
Epoch 4, Train Loss:  1.9778, Train Accuracy: 31.50000000%,Test Loss:  1.9869, Test Accuracy:  24.00000000%
Epoch 5, Train Loss:  1.8453, Train Accuracy: 33.50000000%,Test Loss:  2.0436, Test Accuracy:  28.00000000%
Epoch 6, Train Loss:  1.8316, Train Accuracy: 34.50000000%,Test Loss:  2.0623, Test Accuracy:  20.00000000%
Epoch 7, Train Loss:  1.9084, Train Accuracy: 30.00000000%,Test Loss:  1.9158, Test Accuracy:  34.00000000%
Epoch 8, Train Loss:  1.7199, Train Accuracy: 36.00000000%,Test Loss:  2.0415, Test Accuracy:  24.00000000%
Epoch 9, Train Loss:  1.5388, Train Accuracy: 42.00000000%,Test Loss:  1.9749, Test Accuracy:  38.00000000%
Epoch 10, Train Loss:  1.532